In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

import time
import re
import pandas as pd

from tqdm import tqdm
from collections import defaultdict

from bs4 import BeautifulSoup
import requests


## Что нужно собрать:
* Гиперссылка на проект
* Название проекта
* Организация
* Конкурс
* Запрашиваемая сумма
* Cофинансирование
* Общая сумма расходов на реализацию проекта
* Цель
* Краткое описание
* Статус проекта
* Номер заявки (индекс таблицы)
* Дата подачи
* Сроки реализации
* ИНН
* ОГРН

In [120]:
start_url = 'https://президентскиегранты.рф/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=36568399-3aff-45f4-b418-d5362caf4595&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false'

In [77]:
browser = webdriver.Chrome(ChromeDriverManager().install(), ) # options=chrome_options
timeout = 10

In [75]:
def scrape_links():
    table_project = browser.find_element_by_xpath("//div[@class='table table--p-present table--projects']")
    project_links = []
    for i in table_project.find_elements_by_tag_name('a'):
        project_links.append(i.get_attribute('href'))
    return project_links

In [143]:
def scrape_elems():
    status = browser.find_element_by_xpath("//p[@class='winner-info__status']//span").text
    title = browser.find_element_by_tag_name('h2').text
    
    # сбор информации в content winner
    info = browser.find_element_by_xpath("//ul[@class='winner-info__list winner-info__item']")
    info_elems = info.find_elements_by_xpath("//span[@class='winner-info__list-item-text']")
    
    competition_year = info_elems[0].text
    grant_direction = info_elems[1].text
    app_number = info_elems[2].text
    app_date = info_elems[3].text
    implementation_terms = info_elems[4].text
    organization = info_elems[5].text
    inn = info_elems[6].text
    ogrn = info_elems[7].text
    
    # сбор информации с кратким описанием проекта
    summary = browser.find_element_by_xpath("//div[@id='winner-summary']").text.split('\n')[1]
    target = browser.find_element_by_xpath("//div[@id='winner-aims']").find_element_by_tag_name('li').text
    
    return status, title, competition_year, grant_direction, app_number, app_date, \
            implementation_terms, organization, inn, ogrn, summary, target

In [159]:
proj_data = []

browser.get(start_url)
WebDriverWait(browser, timeout).until(
                    lambda browser: browser.find_element(By.XPATH, "//div[@class='table table--p-present table--projects']"))

time.sleep(2)
links = scrape_links()

for link in tqdm(links):
    browser.get(link)
    WebDriverWait(browser, timeout).until(
                    lambda browser: browser.find_element(By.XPATH, "//div[@id='winner-aims']"))
    time.sleep(2)
    data = scrape_elems()
    proj_data.append(data)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:17<00:00,  3.89s/it]


In [160]:
cols = ['status', 'title', 'competition_year', 'grant_direction', 'app_number', 'app_date', \
            'implementation_terms', 'organization', 'inn', 'ogrn', 'summary', 'target']

In [162]:
df = pd.DataFrame(proj_data, columns=cols)

In [158]:
type(proj_data[0][0])

tuple

In [122]:
links = scrape_links()

In [131]:
browser.get(links[0])

In [144]:
scrape_elems()

('проект не получил поддержку',
 'Проект по Социо-культурная адаптация детей мигрантов "ВсеВместе"',
 'Второй конкурс 2022',
 'Укрепление межнационального и межрелигиозного согласия',
 '22-2-012794',
 '14.03.2022',
 '01.08.2022 - 01.12.2022',
 'ЗАБАЙКАЛЬСКОЕ РЕГИОНАЛЬНОЕ ОТДЕЛЕНИЕ ОБЩЕРОССИЙСКОЙ ОБЩЕСТВЕННОЙ ОРГАНИЗАЦИИ МАЛОГО И СРЕДНЕГО ПРЕДПРИНИМАТЕЛЬСТВА "ОПОРА РОССИИ"',
 '7536114997',
 '1107500000772',
 'Социо-культурная адаптация детей мигрантов, проживающих в Забайкальском крае средствами русского языка, истории и культуры в условиях образовательных организаций',
 'Создание, внедрение и отработка регламента, программы и алгоритма минимальных и максимальных необходимых действий со стороны как руководящего, так и педагогического состава образовательного учреждения при зачислении ребенка с миграционной историей в состав учащихся. Проведение комплекса мероприятий, направленных на максимально эффективное, быстрое, полноценное и комфортное введение детей с миграционной историей в учебн

In [142]:
browser.find_element_by_xpath("//div[@id='winner-aims']").find_element_by_tag_name('li').text

'Создание, внедрение и отработка регламента, программы и алгоритма минимальных и максимальных необходимых действий со стороны как руководящего, так и педагогического состава образовательного учреждения при зачислении ребенка с миграционной историей в состав учащихся. Проведение комплекса мероприятий, направленных на максимально эффективное, быстрое, полноценное и комфортное введение детей с миграционной историей в учебный процесс и социализацию в ученический коллектив с целью профилактики конфликтов на межнациональной почве, ксенофобии, сохранение межэтнического мира в Забайкальском крае.'

In [36]:
def scrape_project_data(links):
    for link in links:
    browser.get(link)
    WebDriverWait(browser, timeout).until(
                    lambda browser: browser.find_element(By.XPATH, "//div[@id='winner-aims']"))
    time.sleep(2)
    title = browser.find_element_by_tag_name('h2').text
    summary = browser.find_element_by_xpath("//div[@id='winner-summary']").text
    target = browser.find_element_by_xpath("//div[@id='winner-aims']").find_element_by_tag_name('li').text
    
    print(f'Название проекта: {title} \nЦель проекта: {target} \nОписание проекта: {summary}')
    print(target)
    print(' ')

In [79]:
project_links = scrape_links()

In [71]:
browser = webdriver.Chrome(ChromeDriverManager().install(), ) # options=chrome_options
time.sleep(3)
for link in proj_links:
    browser.get(link)
    WebDriverWait(browser, timeout).until(
                    lambda browser: browser.find_element(By.XPATH, "//div[@id='winner-aims']"))
#     time.sleep(10)
#     title = browser.find_element_by_xpath("//div[@class='winner-info__title']")
#     print(title)
    title = browser.find_element_by_tag_name('h2').text
    summary = browser.find_element_by_xpath("//div[@id='winner-summary']").text
    target = browser.find_element_by_xpath("//div[@id='winner-aims']").find_element_by_tag_name('li').text
    
    print(f'Название проекта: {title} \nЦель проекта: {target} \nОписание проекта: {summary}')
    print(target)
    print(' ')


Название проекта: Создание адаптивных мастерских для детей с ОВЗ от 7 до 12 лет. 
Цель проекта: Создание образовательной среды по занимательному обучению гончарному, столярному и швейному мастерству детей с ОВЗ с младшего школьного возраста 
Описание проекта: Краткое описание
Проект создания адаптивных мастерских ля детей с ОВЗ будет реализован для целевой группы учеников Православной школы - детей с ОВЗ, которые посещают Православную школу в форме группы продленного дня. Возраст детей - от 7 до 12 лет. Сегодня дети получают в школе присмотр в две смены (либо после школы либо перед школой в зависимости от смены), также организованы уроки по развитию детей с ОВЗ - Правополушарное рисование, шумовой оркестр, занимательная кулинария и музыкальный театр. Дети получают дополнительную подготовку по школьном программе на интерактивном скалодроме, оздоравливаются в соляной пещере. Все это создано в Православной школе с 2018 года по 2021 год при поддержке Фонда Президентских грантов, а также п

Название проекта: Центр Развития Социальных Компетенций 
Цель проекта: Развитие надпрофессиональных компетенций (soft skills) у 1000 студентов из 10 колледжей ДНР И ЛНР через создание Центров Развития Социальных Компетенций и проведение межрегионального чемпионата «Soft Skills Russia» к июлю 2023 года. 
Описание проекта: Краткое описание
Проект Центр Развития Социальных Компетенций - направлен на создание инфраструктуры, развитие, поддержку местных сообществ с целью создания благоприятных условий для развития и самореализации студентов учреждений среднего профессионального образования (СПО) Донецкой Народной Республики и Луганской Народной Республики.
В ходе проекта будет открыто 10 Центров Развития Социальных Компетенций, задействовано 1000 студентов. На открытии центров будут привлекаться сертифицированные тренеры, по направлению надпрофессиональные компетенции (soft skills). В каждом центре будет создана инфраструктура - презентационное оборудование для проведения образовательных со

Название проекта: Вынужденные переселенцы: помощь всем и каждому 
Цель проекта: Содействие в адаптации, стабилизации и социализации семей вынужденных переселенцев 
Описание проекта: Краткое описание
Наша организация на протяжении 5 лет оказывает адресную помощь кризисным и малоимущим семьям, а еще, в настоящее время, в условиях сложной ситуации, сложившейся в ДНР и ЛНР, помогает вынужденным переселенцам с Донбасса и Украины, ведь вынужденно уехали в основном женщины и дети и им сейчас очень сложно. Данный проект направлен на оказание помощи семьям вынужденных переселенцев на этапе адаптации в новых социально-экономических условиях на новом месте переселения в условиях трудной жизненной ситуации.
Для решения этих вопросов мы планируем провести:
1. Индивидуальное и групповое психологическое консультирование семей с детьми вынужденных переселенцев, не менее 100 семей. Занятия будут построены на интегративной психотерапевтической модели психологического сопровождения семей с детьми, помощи

Название проекта: Шаг в будущее 
Цель проекта: Содействовать успешному обучению особенных детей в общеобразовательных школах г. Хабаровска через повышение профессиональных компетенций педагогов, работающих в инклюзии, и на основе пилотной апробации модели по комплексной подготовке детей с РАС к школе с применением метода прикладного анализа поведения и организацией их досуговой деятельности 
Описание проекта: Краткое описание
Проект направлен на создание условий (запуск тренировочной площадки для комплексной подготовки к школе, обучение педагогов прикладному анализу поведения) для успешного обучения детей с расстройством аутистического спектра (далее РАС) в общеобразовательных школах города Хабаровска. К реализации проекта будут привлечены профессиональные куратор и супервизор. Благополучателями проекта являются 12 детей с РАС 6-8 лет. Дети будут поделены на 3 группы по 4 ребенка. Посещение групповых занятий ребенку с аутизмом необходимо для выработки своеобразного «стереотипа» учебног

Название проекта: Оптимизация системы оказания неотложной социально-бытовой помощи людям, оказавшимся в г. Москве без крова и средств 
Цель проекта: Повышение качества и увеличение количества фактов оказания комплексной социальной и бытовой помощи в условиях значительного роста количества обращений людей, оказавшихся без крова и средств на улицах Москвы, посредством внедрения технологических инноваций в организацию процесса оказания неотложной бытовой и срочной социальной помощи 
Описание проекта: Краткое описание
РОО "Милосердие" реализует проект "Служба помощи бездомным" 11 год. До 2020 г. в день фиксировалось в среднем 100 обращений за помощью от людей, оставшихся без крова и средств на улицах Москвы. В 2021 г. в Службу помощи бездомным поступило 142 958 обращений за бытовой помощью и 3 226 обращений за социальной помощью. Зимой 2022 г. количество обращений в день возросло до 500, что свидетельствует о вовлечении в среду бездомных новых людей, среди них много тех, кто потерял работу

Название проекта: Инклюзивная спортивная суббота 
Цель проекта: Проект направлен на обучение и консультирование родителей, создание условий по повышению их физкультурно-оздоровительной компетенции по вопросам организации двигательной активности и вовлечение семей в регулярные занятия по адаптивной-физической культуре в зимние виды спорта, легкой атлетике и игровых видов спорта. 
Описание проекта: Краткое описание
Проект предполагает разработку и реализацию обучающего курса для родителей детей с инвалидностью, в том числе с ментальной инвалидностью, создание условий по вопросам воспитания и психолого-педагогического сопровождения образовательной и двигательной деятельности, как в зимний период (занятия по горнолыжному спорту; сноуборду и бегу на снегоступах, так и в летний период) Организацией самостоятельной физической активности в семейных
условиях, ведение здорового образа жизни. Обучение будут проводить квалифицированные специалисты в области коррекционной педагогики, психологии и а

In [65]:
browser.find_element_by_xpath("//div[@class='winner-info__title']")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='winner-info__title']"}
  (Session info: chrome=105.0.5195.102)


In [66]:
browser.find_element_by_tag_name('h2').text

'Просто настало время помогать'